## Deploying model as a service

- For the sake of this tutorial we will be deploying the previously built model (in part3 of the example) to Azure Cloud using Azure Cloud Instance (ACI)
- Most of the steps are common for almost all could service providers
- Important thing to note is the scoring script (score.py) which defines `request/response` schema needed for Certifai  scan


### In the section below we will:

1. Configure Azure workspace
2. Register model (built in part3 of the example) to the workspace
3. Create a prediction environment in the remote Azure workspace (created above) and
4. Deploy model (predict) as web service

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the 
[configuration-notebook](https://github.com/Azure/MachineLearningNotebooks/blob/c520bd1d4130d9a01ee46e0937459e2de95d15ec/configuration.ipynb) to create an Azure workspace. Creating remote environments/dependencies will be covered in the notebook

**PleaseNote**: to step through this notebook, make sure you have necessary dependencies installed locally

- python>=3.6.2,<3.7
- scikit-learn>0.23.1
- numpy=1.19.1
- pandas
- azureml-sdk=1.12.0 (pip install azureml-sdk==1.12.0)
- matplotlib
- jupyter

In [1]:
import joblib
import numpy as np
import pandas as pd

### Configure and Initialize Azure workspace

- Follow the instructions listed here [creating and managing azure-ml workspace](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace) to create an azure-ml workspace

**Once you have the workspace created easiest way to run through remaining steps is to download the `config.json` to the current directory and replace the exisiting config.json**

### Create a [Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) object from the persisted configuration.

In [2]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [3]:
# deps test model for model serialization/de-serialization
# model was built with scikit-learn 0.23.2 

import sklearn as sklearn_version_test
assert sklearn_version_test.__version__ >= '0.23.1', 'scikit-learn version mismatch, `pip install scikit-learn>=0.23.1` to install right sklearn version for this notebook'
assert np.__version__                   >= '1.16.2' , 'numpy version mismatch, `pip install numpy>=1.16.2` to install right numpy version for this notebook'

In [4]:
# load model 
model_path = 'models/german_credit_multiclass.joblib'
print(f'loading {model_path}')
try:
    model = joblib.load(model_path)
except FileNotFoundError as e:
    print(f'model `{model_path}` not found. Looks like model has not been trained or file location is wrong')
    raise Exception(str(e))
print(model)

loading models/german_credit_multiclass.joblib
Pipeline(steps=[('full_pipeline',
                 Pipeline(steps=[('scaler', StandardScaler())])),
                ('model',
                 GridSearchCV(cv=5, estimator=LogisticRegression(),
                              param_grid={'C': (0.5, 1.0, 2.0),
                                          'max_iter': [1000],
                                          'solver': ['lbfgs']}))])


### Register model to created  workspace

- Register a file or folder as a model by calling [Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-).

- In addition to the content of the model file itself (model + scaler object), our registered model will also store model metadata like model description, tags, etc. -- that will be useful when managing and deploying models in our workspace.

In [5]:
from azureml.core.model import Model

target_en_multiclass_german_credit = Model.register(model_path=model_path,
                       model_name='german_credit_target_encoded_multiclass',
                       tags={'area': "banking credit risk", 'type': "multi-class"},
                       description="Logistic Classifier model to predict credit loan approved/denied/further Inspection",
                       workspace=ws)

Registering model german_credit_target_encoded_multiclass


### Create a custom prediction environment inside azure-ml workspace

If we want control over how our model is run, if it uses another framework, or if it has special runtime requirements, we can instead specify our own environment and scoring method. Custom environments can be used for any model we want to deploy.

Specify the model's runtime environment by creating an [Environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) object and providing the [CondaDependencies](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) needed by the model

In this example we will create a conda environment for our german credit model from file **myenv.yml** and register it to our workspace


In [6]:
with open("myenv.yml", 'r') as f:
    print(f.read())

name: project_environment
dependencies:
  - python=3.6.2
  - scikit-learn>=0.23.1
  - numpy>=1.16.2
  - joblib
  - pip:
    - azureml-defaults
    - inference-schema[numpy-support]


In [7]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

environment = Environment("target-en-multiclass")
environment.python.conda_dependencies = CondaDependencies("myenv.yml")
environment.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200723.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "target-en-multiclass",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
             

## Certifai model invoke request/response schema

Make a note of the request/response schema in `score.py`

- Certifai invokes model with the json schema:

```
{
	"payload": {
		"instances": [
			[6, 107, 88, 0, 0, 36.8, 0.727, 31],
			[5, 100, 80, 0, 0, 31.9, 0.61, 33]
		]
	}
}
```

**where individual list of values correspond to a `row` in the dataset**

- Certifai expects model responses with the json schema: 

```
{
	"payload": {
		"predictions": [1, 0]
	}
}
```

**where `predictions` correspond to an ordered list of model predict responses**

**Important**: Certifai needs batch-predictions enabled for serving models in-order to be performant. It invokes model  predicts with batches of size 4K.


## Create Inference Configuration and deploy webservice

**Inference Configuration** will contain:

1. Scoring script
2. Environment (created above)

We create the scoring script, called **score.py**. The web service call uses this script to show how to use the model.

We include below two required functions in the scoring script:

1. The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started.

2. The `run(data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are also supported.



In [8]:
!cat scripts/score.py

import os
import json
import numpy as np
import joblib
import traceback


def init():
	global model
	# AZUREML_MODEL_DIR is an environment variable created during deployment.
	# It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
	# For multiple models, it points to the folder containing all deployed models (./azureml-models)
	model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'german_credit_multiclass.joblib')
	# deserialize the model_obj file back into a sklearn model and scaler object
	model  = joblib.load(model_path)
	print(model)


def run(data):
	try: 
		# certifai invokes model with the json schema -> {"payload": {"instances": [ [6,107,88,0,0,36.8,0.727,31], [5,100,80,0,0,31.9,0.61,33] ]}}
		data  = json.loads(data).get('payload', {}).get('instances', [])
		data  = np.array(data, dtype=object)
		data  = data if data.ndim == 2 else np.reshape(data, (1, -1))
		result = model.predict(data)
		# you can return any datatype as long as it is JSON-serializa

Deploy the registered model in the custom environment by providing an [InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) object to [Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-). In this case we are also using the [AciWebservice.deploy_configuration()](https://docs.microsoft.com/python/api/azureml-core/azureml.core.webservice.aci.aciwebservice#deploy-configuration-cpu-cores-none--memory-gb-none--tags-none--properties-none--description-none--location-none--auth-enabled-none--ssl-enabled-none--enable-app-insights-none--ssl-cert-pem-file-none--ssl-key-pem-file-none--ssl-cname-none--dns-name-label-none--) method to generate a custom deploy configuration
        
**Note**: This step can take several minutes.

In [9]:
from azureml.core.model import InferenceConfig
from azureml.core import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice

inference_config= InferenceConfig(entry_script="score.py",
                                   environment=environment,source_directory="scripts")

service_name = 'te-multiclass-gc-service'
aci_deployment_config = AciWebservice.deploy_configuration(auth_enabled=False)


In [10]:
service = Model.deploy(ws, service_name, [target_en_multiclass_german_credit],inference_config=inference_config,
                       deployment_config=aci_deployment_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Running....
Succeeded
ACI service creation operation finished, operation "Succeeded"


## Test the webservice

- create the data instances to test with
- invoke the service endpoint

In [11]:
import json
base_path = '../..'
all_data_file = f"{base_path}/datasets/german_credit_eval_multiclass_encoded.csv"
df = pd.read_csv(all_data_file)

X_test = df.drop('outcome',axis=1).values

sample_input = json.dumps({
"payload": {
    "instances": 
        X_test[:10].tolist()
}
})

In [12]:
import requests
import json

headers = {
    'Content-Type': 'application/json'
          }

response = requests.post(
    service.scoring_uri, data=sample_input, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

200
0:00:02.433177
{'payload': {'predictions': [1, 2, 1, 1, 1, 1, 1, 1, 1, 1]}}


In [13]:
local_scan_definition_file = 'target_encoded_gcredit_multiclass_scan_def.yaml'
import yaml

with open(local_scan_definition_file) as file:
    scan_def = yaml.load(file, Loader=yaml.FullLoader)

### Update scan definition yaml

- update the model `predict-endpoint` with the scoring uri of the deployed web service
- add the auth header deatils (if auth enabled in service)
- save the scan definition yaml for remote scanning

In [14]:
# update the predict-endpoint with scoring uri
scan_def['models'][0]['predict_endpoint'] = service.scoring_uri

In [15]:
# add header details
scan_def['model_headers'] = {}
scan_def['model_headers']['default'] = [{'name': 'Content-Type', 'value':'application/json'},
                                        {'name': 'accept',        'value':'application/json'} ]

'''if web-service auth is enabled un-comment the below code snippet'''


# scan_def['model_headers']['defined'] = [{'model_id': 'german_credit_multiclass',
#                                          'name': 'Authorization', 
#                                          'value':'Bearer <INSERT_TOKEN_HERE>'}]


'if web-service auth is enabled un-comment the below code snippet'

In [16]:
# save yaml to disk
with open(local_scan_definition_file, 'w') as file:
    yaml.dump(scan_def, file)